# Poleward CDW transport across different canyon topographies


In [1]:
%matplotlib inline
#%config InlineBackend.figure_format='retina'

In [2]:
from matplotlib import rc
rc('font', **{'size':20})
rc('text', usetex=True)
rc('xtick', labelsize=26) #20 
rc('ytick', labelsize=26) #20
rc('axes', titlesize=26)    # fontsize of the axes title
rc('axes', labelsize=26) #24    # fontsize of the x and y labels

import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import os
import xarray as xr
from xgcm import Grid

In [3]:
## animation block preamble
from matplotlib.animation import FuncAnimation
import IPython
from IPython.display import HTML, display, clear_output

In [4]:
nlayers = 4

In [5]:
from dask.distributed import Client

c=Client(local_directory = '/scratch/v45/qo9901/') 
c

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 6
Total threads: 24,Total memory: 200.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45901,Workers: 6
Dashboard: /proxy/8787/status,Total threads: 24
Started: Just now,Total memory: 200.00 GiB
Comm: tcp://127.0.0.1:37771,Total threads: 4
Dashboard: /proxy/37055/status,Memory: 33.33 GiB
Nanny: tcp://127.0.0.1:39951,


In [6]:
from matplotlib import ticker, cm

### Eddy flux using daily diagnostics -- currently 15 days average

Aim to calculate $\bar{v'h'}$  from $\bar{v'h'} = \bar{vh} - \bar{v}\bar{h}$. 

Diagnostics obtained daily for three years

$\bar{vh}$: from $vh$ diagnostc 

$\bar{v}\bar{h}$: from $v$ and $h$ diagnostic 

Eddy fluxes are the perturbation from the time mean $\bar{v\prime h \prime} = \bar{vh} - \bar{v}\bar{h}$

from $vh = \bar{vh} + v\prime h\prime$

$\bar{vh} = \bar{v}\bar{h} + \bar{\bar{v}h\prime} + \bar{\bar{h}v\prime} + \bar{v\prime h\prime}$ 
$ = \bar{v}\bar{h} + \bar{v\prime h\prime}$


#### Poleward transport in each layer
To isolate the time-meaned poleward transport, we use the quantity $\bar{vh}$, but mask out the positive values, integrating over only the negative values. 

Therefore the poleward transport is $\sum \bar{vh} \hspace{2mm} dx$ if $\bar{vh}<0$  where $dx = 1000m$.

To scale with canyon width, we divide the poleward transport by the canyon width. 
 
 $(\sum \bar{vh} \hspace{2mm} dx$ if $\bar{vh}<0$ )/$\sigma_{canyon}$ 

In [11]:
def polewardtransport_inlayers(filename, nlayers, t1,t2, file):

    ''' Function to compute the poleward transport in exp in each layer
    INPUTS:
    filename: directory of experiment as str
    nlayers:
    
    OUTPUTS:
    transport_array: array of residual poleward transport in each layer $\bar{vh} dx$
    '''
    #os.chdir('/scratch/v45/qo9901/mom6/archive/' + filename)

    if file == 'daily' :   
        FileList = ['/scratch/v45/qo9901/mom6/archive/' + filename + '/output' + '{0:03}'.format(i) + '/prog_daily.nc' for i in range(t1,t2)]
        print('daily')
        print(FileList)
    elif file == 'chunked':
        FileList = ['/scratch/v45/qo9901/mom6/archive/' + filename + '/output' + '{0:03}'.format(i) + '/prog_chunked.nc' for i in range(t1,t2)]
        print('chunked')
        print(FileList)
        
    print('loading...')
    dsx_prog_daily = xr.open_mfdataset(FileList, combine='by_coords',  chunks={"Time": 1000},decode_times=False, engine='netcdf4', parallel=True, drop_variables = {'u','v','e','h'}) #, decode_times=False,
    dsx_prog_daily.coords['Time'] = dsx_prog_daily.coords['Time']/(365)
    print('loaded')
    ## new calculation, dropping negatives then summing
    slice_vh = (dsx_prog_daily.vh.isel(zl = 2).sel(yq = -100, method = 'nearest'))
    print('slice_vh')
    slice_neg = slice_vh.where(slice_vh<0, drop = True)
    print('slice_neg')
    transport_array = ((slice_neg).sum(dim = 'xh')).mean(dim = 'Time')
    print(dsx_prog_daily.vh)
    print('write transport_array')
    del dsx_prog_daily
    del slice_vh, slice_neg
    
    
    return transport_array

In [9]:
import dask 
dask.config.set(**{'array.slicing.split_large_chunks': False})

In [ ]:
polewardtransport_inlayers('4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_sig1p0e+01', 4,80,130,'chunked')

In [ ]:
# save for gaussian shaped canyons

In [17]:
import netCDF4 as nc

fn = '/g/data/v45/qo9901/poleward_gaussian.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

In [18]:
canyon_exp = ds.createDimension('canyon_width',6)
canyon_exps = ds.createVariable('canyon_width', 'f4', ('canyon_width',))
value = ds.createVariable('value', 'f4', ('canyon_width',))


In [21]:
value[0] = polewardtransport_inlayers('4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_sig1p0e+01', 4,  80,130, 'chunked')
value[1] = polewardtransport_inlayers('4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_sig2p5e+01', 4,  50,100, 'chunked')
value[2] = polewardtransport_inlayers('4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_sig5p0e+01', 4,  50,100, 'daily')   
value[3] = polewardtransport_inlayers('4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_sig7p5e+01', 4,  50,100, 'daily')
value[4] = polewardtransport_inlayers('4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_sig1p0e+02', 4,  80,130,'chunked')
value[5] = polewardtransport_inlayers('4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_sig1p2e+02', 4,  30,80, 'daily')


chunked
['/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_sig1p0e+01/output080/prog_chunked.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_sig1p0e+01/output081/prog_chunked.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_sig1p0e+01/output082/prog_chunked.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_sig1p0e+01/output083/prog_chunked.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_sig1p0e+01/output084/prog_chunked.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_sig1p0e+01/output085/prog_chunked.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_sig1p0e+01/output086/prog_chunked.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_sig1p0e+01/output087/prog_chunked.nc', '/scratch/v45/qo9901/mo

In [22]:
ds.close()

In [ ]:
# save for slope without canyon

In [23]:
fn = '/g/data/v45/qo9901/zero_canyon.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

In [24]:
canyon_exp = ds.createDimension('canyon_width',1)
canyon_exps = ds.createVariable('canyon_width', 'f4', ('canyon_width',))
value = ds.createVariable('value', 'f4', ('canyon_width',))
value[0] = polewardtransport_inlayers('4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_zerocanyon_irregular', 4,  20,70, 'daily')


daily
['/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_zerocanyon_irregular/output020/prog_daily.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_zerocanyon_irregular/output021/prog_daily.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_zerocanyon_irregular/output022/prog_daily.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_zerocanyon_irregular/output023/prog_daily.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_zerocanyon_irregular/output024/prog_daily.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_zerocanyon_irregular/output025/prog_daily.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_zerocanyon_irregular/output026/prog_daily.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILL_nDSW_zerocany

In [25]:
ds.close()

In [ ]:
# save for steep-sided canyons

In [28]:
fn = '/g/data/v45/qo9901/poleward_steep.nc'
ds = nc.Dataset(fn, 'r+', format='NETCDF4')

In [29]:
canyon_exp = ds.createDimension('canyon_width',5)
canyon_exps = ds.createVariable('canyon_width', 'f4', ('canyon_width',))
value = ds.createVariable('value', 'f4', ('canyon_width',))


In [30]:
for i in [0]: #range(3): 
    value[0] = polewardtransport_inlayers('4layer_fresh_tau1p0e-01_newtopo_canyonSILLSTEEP_nDSW_sig1p0e+01', 4, (i+1)*50-1, 30,80, 'daily')
    value[1] = polewardtransport_inlayers('4layer_fresh_tau1p0e-01_newtopo_canyonSILLSTEEP_nDSW_sig2p5e+01', 4, (i+1)*50-1, 30,80, 'daily')
    value[2] = polewardtransport_inlayers('4layer_fresh_tau1p0e-01_newtopo_canyonSILLSTEEP_nDSW_sig5p0e+01', 4, (i+1)*50-1, 30,80, 'daily')   
    value[3] = polewardtransport_inlayers('4layer_fresh_tau1p0e-01_newtopo_canyonSILLSTEEP_nDSW_sig7p5e+01', 4, (i+1)*50-1, 30,80, 'daily')
    value[4] = polewardtransport_inlayers('4layer_fresh_tau1p0e-01_newtopo_canyonSILLSTEEP_nDSW_sig1p0e+02', 4, (i+1)*50-1, 51,101, 'daily')


daily
['/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILLSTEEP_nDSW_sig1p0e+01/output030/prog_daily.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILLSTEEP_nDSW_sig1p0e+01/output031/prog_daily.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILLSTEEP_nDSW_sig1p0e+01/output032/prog_daily.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILLSTEEP_nDSW_sig1p0e+01/output033/prog_daily.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILLSTEEP_nDSW_sig1p0e+01/output034/prog_daily.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILLSTEEP_nDSW_sig1p0e+01/output035/prog_daily.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILLSTEEP_nDSW_sig1p0e+01/output036/prog_daily.nc', '/scratch/v45/qo9901/mom6/archive/4layer_fresh_tau1p0e-01_newtopo_canyonSILLSTEEP_nDSW_sig1p0e+01/output037/prog_daily.nc', '

In [31]:
value

<class 'netCDF4._netCDF4.Variable'>
float32 value(canyon_width)
unlimited dimensions: 
current shape = (5,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [32]:
ds.close()